In [ ]:
'''
author: @ericslevenson
adapted by: @addisonpletcher
date: 11/28/2022
description: GEE script to export near-daily records of lake area within a
shapefile of buffered lakes
'''

'\nauthor: @ericslevenson\ndate: 11/28/2022\ndescription: GEE script to export near-daily records of lake area within a\nshapefile of buffered lakes\n'

### Preliminary

In [6]:
# Authenticate private account (only required for exporting to drive/gee/gcp)
#from google.colab import auth
#auth.authenticate_user()

# Earth Engine setup
import ee # Trigger the authentication flow.
ee.Authenticate()
ee.Initialize() # Initialize the library.

# Google Drive setup (if needed)
from google.colab import drive
drive.mount('/content/drive')

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=QrczaEGh120F4EekvPpYCbcucQ8WA5XG-AcR-k6XTBc&tc=OER_LWr6iNq_PFA1pK5IQtylghDKTaVnNHgG7SjMr4k&cc=9gzapIjKtPEayZtfdG4TSF1fbyrMK_a7mMLYXviq32c

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXmZs_fVXMsoxxeoh8EIB4fhsM1BYyhhZIbt5Yhu1yI4RsLgzZZ4AkM

Successfully saved authorization token.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
## ***INPUTS***

#TODO: can I remove tile stuff here since I'm not working off tiles? or should I work off tiles since I'm in specific UTM zones?

## Regularly Adjusted inputs ##
# Time Period
start = '2018-06-29'
finish = '2018-07-01'
# Tile of interest
tile = 6    # since I'm not running via a tile scheme, can this be removed?
# Lake Shapefile
lakes = ee.FeatureCollection('projects/ee-addisonpletcher/assets/buffered_lakemask') # already a vector w/ buffered features

## Periodically Adjusted Inputs ##
# ROI tile collection
tiles = ee.FeatureCollection('projects/ee-addisonpletcher/assets/test_ROI2')

## Rarely Adjusted inputs ##
# Image scale
pixScale = 10

# ***EXPORTS***
#TODO: remove centroid, add lat/long + Lake_ID. Removed FID

# Export Properties
exportSelectors = ['id', 'Lake_ID', 'waterArea', 'clearArea', 'cloudArea', 'centroid'] # These will likely need to be changed to reflect your input shapefile. Specifically, 'count', 'centroid', and 'ratio' are all attributes from the imported EE asset. The others can stay the same.
# ROI Description
roiLabel = str('harmonized')
# Export Folder
exportFolder = 'YKD_water'
#TODO: reroute export folder to be 'raw_csvs', then also change where visualizations go


# ***EARTH ENGINE-IFY***
eestart = ee.Date(start)
eefinish = ee.Date(finish)
startDoy = ee.Date(start).getRelative('day', 'year')
endDoy = ee.Date(finish).getRelative('day', 'year')
# eeroi = tiles.filter(ee.Filter.eq('id', tile)).first() #don't filter, no tiles
eeroi = tiles.first()
roi = ee.Geometry.Polygon(eeroi.geometry().getInfo()['coordinates'][0])
lakes = lakes.filterBounds(roi) # filter lakes to roi

### Functions

In [8]:
## ***IMAGE PRE-PROCESSING METHODS***

# Mask clouds in Sentinel-2
def maskS2clouds(image):
  '''Takes an input and adds two bands: cloud mask and clear mask'''
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  clear_mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0)).rename('clear_mask')
  cloud_mask = qa.bitwiseAnd(cloudBitMask).eq(1).And(qa.bitwiseAnd(cirrusBitMask).eq(1)).rename('cloud_mask')
  return image.addBands([cloud_mask,clear_mask])

# Clip image
def clip_image(image):
  '''Clips to the roi defined at the beginning of the script'''
  return image.clip(roi)

def clip2lakes(image):
  '''Clips an image based on the lake boundaries'''
  return image.clip(lakes)

# Get percentile cover
def getCover(image):
  '''calculates percentage of the roi covered by the clear mask. NOTE: this function
  calls the global totPixels variable that needs to be calculated in the main script.'''
  actArea = ee.Number(image.updateMask(image.select('B2')).reduceRegion(
      reducer = ee.Reducer.count(),
      scale = 100,
      maxPixels=1e12,
      ).values().get(0)).multiply(10000)
  # calculate the perc of cover OF CLEAR PIXELS
  percCover = actArea.divide(area).multiply(100)
  # number as output
  return image.set('percCover', percCover,'actArea',actArea)

# Mosaic images by date, orbit, - basically combines images together that were taken on the same day
def mosaicBy(imcol):
  '''Takes an image collection (imcol) and creates a mosaic for each day
  Returns: An image collection of daily mosaics'''
  #return the collection as a list of images (not an image collection)
  imlist = imcol.toList(imcol.size())
  # Get all the dates as list
  def imdate(im):
    date = ee.Image(im).date().format("YYYY-MM-dd")
    return date
  all_dates = imlist.map(imdate)
  # get all orbits as list
  def orbitId(im):
    orb = ee.Image(im).get('SENSING_ORBIT_NUMBER')
    return orb
  all_orbits = imlist.map(orbitId)
  # get all spacecraft names as list
  def spacecraft(im):
    return ee.Image(im).get('SPACECRAFT_NAME')
  all_spNames = imlist.map(spacecraft)
  # this puts dates, orbits and names into a nested list
  concat_all = all_dates.zip(all_orbits).zip(all_spNames);
  # here we unnest the list with flatten, and then concatenate the list elements with " "
  def concat(el):
    return ee.List(el).flatten().join(" ")
  concat_all = concat_all.map(concat)
  # here, just get distinct combintations of date, orbit and name
  concat_unique = concat_all.distinct()
  # mosaic
  def mosaicIms(d):
    d1 = ee.String(d).split(" ")
    date1 = ee.Date(d1.get(0))
    orbit = ee.Number.parse(d1.get(1)).toInt()
    spName = ee.String(d1.get(2))
    im = imcol.filterDate(date1, date1.advance(1, "day")).filterMetadata('SPACECRAFT_NAME', 'equals', spName).filterMetadata('SENSING_ORBIT_NUMBER','equals', orbit).mosaic()
    return im.set(
        "system:time_start", date1.millis(),
        "system:date", date1.format("YYYY-MM-dd"),
        "system:id", d1)
  mosaic_imlist = concat_unique.map(mosaicIms)
  return ee.ImageCollection(mosaic_imlist)

###########################################################################
## ***WATER CLASSIFICATION METHODS***

# Define NDWI image
def ndwi(image):
  '''Adds an NDWI band to the input image'''
  return image.normalizedDifference(['B3', 'B8']).rename('NDWI').multiply(1000)

# Basic ndwi classification
def ndwi_classify(image):
  '''Creates a binary image based on an NDWI threshold of 0'''
  ndwimask = image.select('NDWI')
  water = ndwimask.gte(0)
  land = ndwimask.lt(0)
  return(water)

# OTSU thresholding from histogram
def otsu(histogram):
  '''Returns the NDWI threshold for binary water classification'''
  counts = ee.Array(ee.Dictionary(histogram).get('histogram'))
  means = ee.Array(ee.Dictionary(histogram).get('bucketMeans'))
  size = means.length().get([0])
  total = counts.reduce(ee.Reducer.sum(), [0]).get([0])
  sum = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0])
  mean = sum.divide(total)
  indices = ee.List.sequence(1, size)
  def func_xxx(i):
    '''Compute between sum of squares, where each mean partitions the data.'''
    aCounts = counts.slice(0, 0, i)
    aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
    aMeans = means.slice(0, 0, i)
    aMean = aMeans.multiply(aCounts) \
        .reduce(ee.Reducer.sum(), [0]).get([0]) \
        .divide(aCount)
    bCount = total.subtract(aCount)
    bMean = sum.subtract(aCount.multiply(aMean)).divide(bCount)
    return aCount.multiply(aMean.subtract(mean).pow(2)).add(
           bCount.multiply(bMean.subtract(mean).pow(2)))
  bss = indices.map(func_xxx)
  # Return the mean value corresponding to the maximum BSS.
  return means.sort(bss).get([-1])

# OTSU thresholding for an image
def otsu_thresh(water_image):
  '''Calculate NDWI and create histogram. Return the OTSU threshold.'''
  NDWI = ndwi(water_image).select('NDWI').updateMask(water_image.select('clear_mask'))
  histogram = ee.Dictionary(NDWI.reduceRegion(
    geometry = roi,
    reducer = ee.Reducer.histogram(255, 2).combine('mean', None, True).combine('variance', None, True),
    scale = pixScale,
    maxPixels = 1e12
  ))
  return otsu(histogram.get('NDWI_histogram'))

# Classify an image using OTSU threshold.
def otsu_classify(water_image):
  '''(1) Calculate NDWI and create histogram. (2) Calculate NDWI threshold for
  binary classification using OTSU method. (3) Classify image and add layer to input image.
  '''
  NDWI = ndwi(water_image).select('NDWI')
  histogram = ee.Dictionary(NDWI.reduceRegion(
    geometry = roi,
    reducer = ee.Reducer.histogram(255, 2).combine('mean', None, True).combine('variance', None, True),
    scale = pixScale,
    maxPixels = 1e12
  ))
  threshold = otsu(histogram.get('NDWI_histogram'))
  otsu_classed = NDWI.gt(ee.Number(threshold)).And(water_image.select('B8').lt(2000)).rename('otsu_classed')
  return water_image.addBands([otsu_classed])

def adaptive_thresholding(water_image):
  '''Takes an image clipped to lakes and returns the water mask'''
  NDWI = ndwi(water_image).select('NDWI')#.updateMask(water_image.select('clear_mask')) # get NDWI **TURNED OFF CLOUD MASK, SHOULD THIS STAY OFF?**
  threshold = ee.Number(otsu_thresh(water_image))
  threshold = threshold.divide(10).round().multiply(10)
  # get fixed histogram
  histo = NDWI.reduceRegion(
      geometry = roi,
      reducer = ee.Reducer.fixedHistogram(-1000, 1000, 200),
      scale = pixScale, # This was 30, keep at 10!?!?
      maxPixels = 1e12
  )
  hist = ee.Array(histo.get('NDWI'))
  counts = hist.cut([-1,1])
  buckets = hist.cut([-1,0])
  #find split points from otsu threshold
  threshold = ee.Array([threshold]).toList()
  buckets_list = buckets.toList()
  split = buckets_list.indexOf(threshold)
  # split into land and water slices
  land_slice = counts.slice(0,0,split)
  water_slice = counts.slice(0,split.add(1),-1)
  # find max of land and water slices
  land_max = land_slice.reduce(ee.Reducer.max(),[0])
  water_max = water_slice.reduce(ee.Reducer.max(),[0])
  land_max = land_max.toList().get(0)
  water_max = water_max.toList().get(0)
  land_max = ee.List(land_max).getNumber(0)
  water_max = ee.List(water_max).getNumber(0)
  #find difference between land, water and otsu val
  counts_list = counts.toList()
  otsu_val = ee.Number(counts_list.get(split))
  otsu_val = ee.List(otsu_val).getNumber(0)
  land_prom = ee.Number(land_max).subtract(otsu_val)
  water_prom = ee.Number(water_max).subtract(otsu_val)
  #find land and water buckets corresponding to 0.9 times the prominence
  land_thresh = ee.Number(land_max).subtract((land_prom).multiply(ee.Number(0.9)))
  water_thresh = ee.Number(water_max).subtract((water_prom).multiply(ee.Number(0.9)))
  land_max_ind = land_slice.argmax().get(0)
  water_max_ind = water_slice.argmax().get(0)
  li = ee.Number(land_max_ind).subtract(1)
  li = li.max(ee.Number(1))
  wi = ee.Number(water_max_ind).add(1)
  wi = wi.min(ee.Number(199))
  land_slice2 = land_slice.slice(0,li,-1).subtract(land_thresh)
  water_slice2 = water_slice.slice(0,0,wi).subtract(water_thresh)
  land_slice2  = land_slice2.abs().multiply(-1)
  water_slice2 = water_slice2.abs().multiply(-1)
  land_index = ee.Number(land_slice2.argmax().get(0)).add(land_max_ind)
  water_index = ee.Number(water_slice2.argmax().get(0)).add(split)
  land_level = ee.Number(buckets_list.get(land_index))
  water_level = ee.Number(buckets_list.get(water_index))
  land_level = ee.Number(ee.List(land_level).get(0)).add(5)
  water_level = ee.Number(ee.List(water_level).get(0)).add(5)
  #calculate water fraction and classify
  water_fraction = (NDWI.subtract(land_level)).divide(water_level.subtract(land_level)).multiply(100).rename('water_fraction')
  #water_fraction = conditional(water_fraction) #sets values less than 0 to 0 and greater than 100 to 100
  water_75 = water_fraction.gte(75).rename('water_75'); #note, this is a non-binary classification, so we use 75% water as "water"
  all_mask = water_image.select('B2').gt(5).rename('all_mask')
  cloud_mask_ed = water_image.select('clear_mask').neq(1).rename('cloud_mask_ed')
  return water_image.addBands([water_fraction,water_75,NDWI,cloud_mask_ed])
# Apply cloud mask to other bands
def applyMask(image):
  img = image.updateMask(image.select('clear_mask'))
  return img
def binaryImage(image):
  '''takes a multiband image and returns just the binary water_75 band'''
  img = image.select('water_75')
  return img
def waterImage(image):
  '''takes a multiband image and returns just the water fraction band'''
  img = image.select('water_fraction')
  return img

###############################################################################
## ***PROPERTY EXTRACTION METHODS***
def sumWater(lake):
  '''sums the water pixels within a buffered lake polygon and adds the result to the feature'''
  watersum = waterAreaIm.select('area').reduceRegion(
      reducer=ee.Reducer.sum(),
      geometry = lake.geometry(),
      scale = 10,
      maxPixels=1e9
  ).get('area')
  return watersum

def sumClear(lake):
  '''sums the number of clear pixels within a buffered lake polygon'''
  clearsum = lakeIm.select('clear_mask').reduceRegion(
      reducer=ee.Reducer.sum(),
      geometry = lake.geometry(),
      scale = 10,
      maxPixels=1e9
  ).get('clear_mask')
  return clearsum

def getClearArea(lake):
  '''trying what i did with water...see if it works! Will potentialy switch for sumClear'''
  clearArea = clearAreaIm.select('area').reduceRegion(
      reducer=ee.Reducer.sum(),
      geometry = lake.geometry(),
      scale = 10,
      maxPixels=1e9
  ).get('area')
  return clearArea

def sumClouds(lake):
  '''sums the number of clear pixels within a buffered lake polygon'''
  cloudsum = lakeIm.select('cloud_mask_ed').reduceRegion(
      reducer=ee.Reducer.sum(),
      geometry = lake.geometry(),
      scale = 10,
      maxPixels=1e9
  ).get('cloud_mask_ed')
  return cloudsum

def getCloudArea(lake):
  '''trying what i did with water...see if it works! Will potentialy switch for sumClouds'''
  cloudArea = cloudAreaIm.select('area').reduceRegion(
      reducer=ee.Reducer.sum(),
      geometry = lake.geometry(),
      scale = 10,
      maxPixels=1e9
  ).get('area')
  return cloudArea

def sumAll(lake):
  '''sums the number of pixels within a buffered lake polygon'''
  all_mask = lakeIm.select('B2').gt(5).rename('all_mask')
  allsum = all_mask.reduceRegion(
      reducer=ee.Reducer.count(),
      geometry = lake.geometry(),
      scale = 10,
      maxPixels=1e9
      ).get('all_mask')
  return allsum

def troid(lake):
  center = ee.Array(lake.centroid().geometry().coordinates())
  return center

def getID(lake):
  '''get the EarthEngine id'''
  id = ee.Number(lake.id())
  return id

#originally FID, adapting for Lake_ID
def getLake_ID(lake):
  '''get the Lake_ID field from shapefile'''
  Lake_ID = ee.Number(lake.get('Lake_ID'))
  return Lake_ID

####################TEST for above##################
#think its working based on Lake_ID being present in resulting csv, but can (and should) run more manual tests to ensure this is tru
####################TEST for above##################

#def getCount(lake):
#  count = ee.Number(lake.get('count'))
#  return count


def lakeProps(lake):
  water = sumWater(lake)
  #clear = sumClear(lake)
  #clouds = sumClouds(lake)
  #all = sumAll(lake)
  centroid = troid(lake)
  id = getID(lake)
  #count = getCount(lake)
  #label = getLabel(lake)
  Lake_ID = getLake_ID(lake)
  cloudArea = getCloudArea(lake)
  clearArea = getClearArea(lake)
  return ee.Feature(None, {'id': id, 'Lake_ID': Lake_ID, 'waterArea': water, 'clearArea': clearArea,  'cloudArea': cloudArea, 'centroid': centroid})
  # deleted: 'allPixels': all, 'cloudPixels': clouds, 'clearPixels': clear, 'FID': fid
###############################################################################
## ***EXPORT METHODS***
def export_lakes(collection, description, fileNamePrefix, fileFormat, folder, selectors):
  '''Export a feature collection of lake properties to google drive for a given day.'''
  task = ee.batch.Export.table.toDrive(**{
    'collection': collection,
    'description': description,
    'fileNamePrefix': fileNamePrefix,
    'fileFormat': fileFormat,
    'folder': folder,
    'selectors': selectors
  })
  task.start()

## Main

In [9]:
##############################################################################
## *** IMAGE PROCESSING ***
images = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterBounds(roi).filterDate(start,finish).filter(ee.Filter.calendarRange(startDoy, endDoy, 'day_of_year')).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',50)) # Get Images
images_all = mosaicBy(images) # Mosaic images
images_all = images_all.map(maskS2clouds) # Create cloud/clear masks
images_all = images_all.map(clip_image) # Clip to roi
images_all.map(applyMask) # mask other bands for clouds
area = roi.area().getInfo() # Calculate total area
#area = roi.area()
#images_all = images_all.map(lambda image: image.set('total_area', area)) #workaround to remove getInfo function

# Filter by percentile cover
images_all = images_all.map(getCover) # Add percent cover as an image property
images_all = images_all.filterMetadata('percCover','greater_than',10) # remove images covering less than 50% of the ROI)
lakeimages = images_all.map(clip2lakes) # Clip images to buffered lake mask
dates = lakeimages.aggregate_array('system:date').getInfo()
dates

['2018-06-29', '2018-06-30']

In [10]:
###############################################################################
## ***WATER CLASSIFICATION***
lakeimages = lakeimages.map(adaptive_thresholding)

###############################################################################
## ***ITERATE THROUGH DAYS AND EXPORT***
for i, date in enumerate(dates):
  # Get lake properties
  eedate = ee.Date(date) #earthengine date format
  lakeIm = lakeimages.filterDate(eedate).first() # get the appropriate date
  areaIm = lakeIm.pixelArea() # get a pixel area image
  lakeIm = lakeIm.addBands([areaIm]) # add pixel area as a band in the image
  waterAreaIm = areaIm.updateMask(lakeIm.select('water_75')) # mask area image based on water
  cloudAreaIm = areaIm.updateMask(lakeIm.select('cloud_mask_ed')) # mask area image based on clouds
  clearAreaIm = areaIm.updateMask(lakeIm.select('clear_mask')) # mask area image based on clearness
  lakes2 = lakes.map(lakeProps)
  # Export
  exportDate = date.replace('-', '_')
  description = roiLabel +'_'+ exportDate
  fileformat = 'CSV'
  export_lakes(lakes2, description, description, fileformat, exportFolder, exportSelectors)

In [ ]:
test = lakes2.first()
test.getInfo()

## Visualization
###### *** When exporting image need to uncheck "include read only scopes" during authentication ***

#### Classified Image Export (single)

In [ ]:
#Original visualization
single = lakeimages.first().select('water_75', 'cloud_mask', 'clear_mask')

# Cast all bands to UInt 8
single = single.toUint8() #exporting image w/ binary classifications ranging from 0-1, so UInt8

task = ee.batch.Export.image.toDrive(**{
    'image': single,
    'description': 'test_img',
    'folder':'Visualization',
    'fileFormat': 'GeoTIFF',
    'scale': 10,
    'region': roi,
    'maxPixels': 1e12
})
task.start()
import time
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)

#### Classified Image Export (Collection)

In [ ]:
import folium
import time

# Define what bands
bands_to_select = ['water_75', 'cloud_mask', 'clear_mask']

# Iterate through the Image Collection and export each image
image_list = lakeimages.toList(lakeimages.size())
for i in range(image_list.length().getInfo()):
    # Get the current image in the collection
    current_image = ee.Image(image_list.get(i))

    # Select the desired bands and cast to UInt8
    selected_image = current_image.select(bands_to_select).toUint8()

    # Get the date from the image's metadata
    date = ee.Date(current_image.get('system:time_start'))
    date_str = date.format('YYYY-MM-dd').getInfo()  # Format the date as desired

    # Use the date as the description
    description = 'image_' + date_str

    # Define the export task for the current image
    task = ee.batch.Export.image.toDrive(**{
        'image': selected_image,
        'description': description,
        'folder': 'Visualization', #have this foler be mapped within YKD_Water folder
        'fileFormat': 'GeoTIFF',
        'scale': 10,
        'region': roi,
        'maxPixels': 1e12
    })

    # Start the export task for the current image
    task.start()

    # Monitor the task's progress
    print('Exporting task (id: {}) - Image {}'.format(task.id, i))

    # Wait for the task to complete before moving on to the next image
    while task.active():
        print('Polling for task (id: {}).'.format(task.id))
        time.sleep(5)


In [ ]:
single = lakeimages.first().select('water_75', 'cloud_mask', 'clear_mask')
single.getInfo()

{'type': 'Image',
 'bands': [{'id': 'water_75',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1},
   'dimensions': [4, 2],
   'origin': [-165, 60],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'cloud_mask',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1},
   'dimensions': [4, 2],
   'origin': [-165, 60],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'clear_mask',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1},
   'dimensions': [4, 2],
   'origin': [-165, 60],
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}],
 'id': ['2018-06-29', '115.0', 'Sentinel-2B'],
 'properties': {'percCover': 191.45828477625665,
  'actArea': 12999650000,
  'system:time_start': 1530230400000,
  'system:date': '2018-06-29',
  'system:index': '0'}}

In [ ]:
# TRY #1
# not functional because i have a feature collection not img collection

import folium

lakes_collection = ee.ImageCollection('lakes2')

# Define the export parameters
export_params = {
    'folder': 'my_export_folder',
    'scale': 30,  # Set the scale to your desired value
    'fileFormat': 'GeoTIFF',
}

# Iterate through the image collection and export each image
image_list = lakes_collection.toList(lakes_collection.size())
for i in range(image_list.length().getInfo()):
    image = ee.Image(image_list.get(i))
    export_params['image'] = image
    export_params['description'] = 'export_' + str(i)  # Change the description as needed
    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()
